In [1]:
import json
import pandas as pd

with open('telegram_export.json') as f:
    data = json.load(f)

In [2]:
df = pd.DataFrame.from_dict(data['chats']['list'][0]['messages'], orient='columns')[['date', 'text', 'type']]
df = df[df['type']=='message']
df['day'] = df['date'].apply(lambda x: x[:10])

In [3]:
df2 = pd.DataFrame({'count' : df.groupby('day').size()}).reset_index()

In [4]:
df['minute'] = df['date'].apply(lambda x: x[:-3])
tmp = pd.DataFrame({'count' : df.groupby('minute').size()}).reset_index()
tmp['day'] = tmp['minute'].apply(lambda x: x[:10])
df3 = pd.DataFrame({'count' : tmp.groupby('day').size()}).reset_index()

In [5]:
df4 = df3[df3['day'] > '2018-01-01']

In [6]:
import matplotlib
from matplotlib import pyplot
from matplotlib import dates
import datetime
import numpy
import time

def plot(df):
    converted_dates = list(map(datetime.datetime.strptime, df['day'], len(df['day'])*['%Y-%m-%d']))
    x_axis = converted_dates
    formatter = dates.DateFormatter('%Y-%m-%d')
    y_axis = df['count'].as_matrix()

    tList = []
    t = 0
    while t < len(x_axis):
        ti = int(time.mktime(x_axis[t].timetuple()))
        tList.append(ti)
        t = t + 1

    z = numpy.polyfit(tList, y_axis, 1)
    p = numpy.poly1d(z)

    pyplot.plot(x_axis, y_axis, '-')
    pyplot.plot(x_axis, p(tList))
    ax = pyplot.gcf().axes[0] 
    ax.xaxis.set_major_formatter(formatter)
    pyplot.gcf().autofmt_xdate(rotation=25)
    pyplot.show()
    # pyplot.savefig('img2.png')

In [ ]:
plot(df2)